In [ ]:
!pip install azure-mgmt-datafactory

  Using cached azure_mgmt_datafactory-6.1.0-py3-none-any.whl (525 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Using cached azure_common-1.1.28-py2.py3-none-any.whl (14 kB)
  Using cached azure_mgmt_core-1.4.0-py3-none-any.whl (27 kB)
  Using cached azure_core-1.30.1-py3-none-any.whl (193 kB)
  Using cached typing_extensions-4.11.0-py3-none-any.whl (34 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-7e9cad02-859e-4e6a-bcf0-352ba9c35676
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-7e9cad02-859e-4e6a-bcf0-352ba9c35676/bin/python -m pip install --upgrade pip' command.


In [ ]:
!pip install azure-identity

  Using cached azure_identity-1.16.0-py3-none-any.whl (166 kB)
  Using cached msal-1.28.0-py3-none-any.whl (102 kB)
  Using cached msal_extensions-1.1.0-py3-none-any.whl (19 kB)
  Using cached PyJWT-2.8.0-py3-none-any.whl (22 kB)
  Using cached portalocker-2.8.2-py3-none-any.whl (17 kB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-7e9cad02-859e-4e6a-bcf0-352ba9c35676/bin/python -m pip install --upgrade pip' command.


In [ ]:
from azure.identity import DefaultAzureCredential, ClientSecretCredential
from azure.mgmt.datafactory import DataFactoryManagementClient
import pandas as pd
import json

In [ ]:
# Configurar credenciais
credentials = ClientSecretCredential(client_id='', client_secret='', tenant_id='') 

# Substitua os valores abaixo com suas informações
subscription_id = ''
resource_group_name = ''
data_factory_name = ''

# Inicializar o cliente do Azure Data Factory
adf_client = DataFactoryManagementClient(credentials, subscription_id)
pipelines = adf_client.pipelines.list_by_factory(resource_group_name, data_factory_name)

In [ ]:
pipe_id = 0
dados_piplines = []
dados_inputs = []
dados_outputs = []
for pipeline in pipelines:
    pipe_id += 1
    nome_pipeline = pipeline.name
    descricao = pipeline.description

    dados_dict = {
        "ID": pipe_id,
        "NOME": nome_pipeline,
        "DESCRICAO": descricao}
    dados_piplines.append(dados_dict)

    pipeline_dict = pipeline.serialize()

    response = adf_client.pipelines.get(resource_group_name, data_factory_name, pipeline_name=f"{nome_pipeline}")

    activities = response.serialize()
    for activity in activities["properties"]["activities"]:
        act_name = activity['name']
        act_inputs = activity.get('inputs', [])
        act_outputs = activity.get('outputs', [])
        query = activity.get('typeProperties', {}).get('source', {}).get('sqlReaderQuery', '')

        active_info_origem = {
            "ID": pipe_id,
            "NOME": act_name,
            "TIPO" : act_inputs[0]["type"] if act_inputs else '',
            "NOME_REFERENCIA" : act_inputs[0]["referenceName"] if act_inputs else '',
            "PARAMETROS" : act_inputs[0]["parameters"] if act_inputs else None,
            "QUERY": query
        }
        active_info_coletor = {
            "ID": pipe_id,
            "NOME": act_name,
            "TIPO" : act_outputs[0]["type"] if act_outputs else '',
            "NOME_REFERENCIA" : act_outputs[0]["referenceName"] if act_outputs else '',
            "PARAMETROS" : act_outputs[0]["parameters"] if act_outputs else ''
        }
        
        dados_inputs.append(active_info_origem)
        dados_outputs.append(active_info_coletor)

In [ ]:
dados_inputs_json = json.dumps(dados_inputs)
dados_outputs_json = json.dumps(dados_outputs)
dados_piplines_json = json.dumps(dados_piplines)

In [ ]:
df = pd.read_json(dados_piplines_json)
display(df)

ID,NOME,DESCRICAO
1,PL_copy_csv_db_cadastros,Cópia do csv -datalake para o banco sql
2,PL_copy_csv_produtos_vendas,null
3,copia_tabela_schema_dbo,Copia tabela de vendas do schema empresa para o schema dbo


In [ ]:
df_inp = pd.read_json(dados_inputs_json)
display(df_inp) 

/databricks/spark/python/pyspark/sql/pandas/conversion.py:467: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  A field of type StructType expects a pandas.DataFrame, but got: <class 'pandas.core.series.Series'>
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


ID,NOME,TIPO,NOME_REFERENCIA,PARAMETROS,QUERY
1,copia csv cliente,DatasetReference,DL_dinamico_virgula,Map(file_name -> CadastroClientes.csv),
1,copia csv - Lojas,DatasetReference,DL_dinamico_virgula,Map(file_name -> Contoso - Lojas.csv),
1,copia csv -funcionarios,DatasetReference,DL_dinamico_ponto_virgula,Map(file_name -> CadastroFuncionarios.csv),
2,copia csv produtos,DatasetReference,DL_dinamico_ponto_virgula,Map(file_name -> Contoso - Cadastro Produtos.csv),
2,inner vendas,DatasetReference,sql_dinamico,"Map(table_name -> vendas, schema_name -> empresa)","SELECT v.[Numero da Venda], v.[Data da Venda], v.[Quantidade Vendida], v.[Quantidade Devolvida], p.[Nome da Marca], p.Tipo, p.Categoria, p.[Preco Unitario], l.[Nome da Loja] FROM empresa.vendas v LEFT JOIN empresa.lojas l ON l.[ID Loja] =v.[ID Loja] LEFT JOIN empresa.produtos p ON p.[ID Produto] = v.[ID Produto]"
3,copy_tabela_vendas_schema_dbo,DatasetReference,sql_dinamico,"Map(table_name -> vendas, schema_name -> empresa)","SELECT v.[Numero da Venda], v.[Data da Venda], v.[Quantidade Vendida], v.[Quantidade Devolvida], p.[Nome da Marca], p.Tipo, p.Categoria, p.[Preco Unitario], l.[Nome da Loja] FROM empresa.vendas v LEFT JOIN empresa.lojas l ON l.[ID Loja] =v.[ID Loja] LEFT JOIN empresa.produtos p ON p.[ID Produto] = v.[ID Produto]"


In [ ]:
df_out = pd.read_json(dados_outputs_json)
display(df_out)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:467: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  A field of type StructType expects a pandas.DataFrame, but got: <class 'pandas.core.series.Series'>
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


ID,NOME,TIPO,NOME_REFERENCIA,PARAMETROS
1,copia csv cliente,DatasetReference,sql_dinamico,"Map(table_name -> clientes, schema_name -> empresa)"
1,copia csv - Lojas,DatasetReference,sql_dinamico,"Map(table_name -> lojas, schema_name -> empresa)"
1,copia csv -funcionarios,DatasetReference,sql_dinamico,"Map(table_name -> funcionarios, schema_name -> empresa)"
2,copia csv produtos,DatasetReference,sql_dinamico,"Map(table_name -> produtos, schema_name -> empresa)"
2,inner vendas,DatasetReference,sql_dinamico,"Map(table_name -> juncao_vendas, schema_name -> empresa)"
3,copy_tabela_vendas_schema_dbo,DatasetReference,AzureSqlTable_dbo_clientes,Map()
